# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType

In [14]:
spark = SparkSession \
    .builder \
    .appName("Data Wrangling Quiz") \
    .getOrCreate()

df = spark.read.json("data/sparkify_log_small.json")

# Question 1

Which page did user id "" (empty string) NOT visit?

In [15]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [16]:
blanks = df.where(df.userId == "").select(col("page")).alias("blank").dropDuplicates()
allpages = df.select("page").drop_duplicates()

for row in set(allpages.collect()) - set(blanks.collect()):
    print(row.page)
    

NextSong
Error
Submit Upgrade
Upgrade
Logout
Submit Downgrade
Save Settings
Downgrade
Settings


# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


# Question 3

How many female users do we have in the data set?

In [17]:
df.where(df.gender == "F").select("userID").dropDuplicates().count()


462

# Question 4

How many songs were played from the most played artist?

In [18]:
df.filter(df.page == "NextSong").select("Artist").groupBy("Artist").agg({"Artist": "count"}).sort(desc("count(Artist)")).show()

+--------------------+-------------+
|              Artist|count(Artist)|
+--------------------+-------------+
|            Coldplay|           83|
|       Kings Of Leon|           69|
|Florence + The Ma...|           52|
|            BjÃÂ¶rk|           46|
|       Dwight Yoakam|           45|
|       Justin Bieber|           43|
|      The Black Keys|           40|
|         OneRepublic|           37|
|                Muse|           36|
|        Jack Johnson|           36|
|           Radiohead|           31|
|        Taylor Swift|           29|
|Barry Tuckwell/Ac...|           28|
|          Lily Allen|           28|
|               Train|           28|
|           Daft Punk|           27|
|           Metallica|           27|
|          Nickelback|           27|
|          Kanye West|           26|
|Red Hot Chili Pep...|           24|
+--------------------+-------------+
only showing top 20 rows



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [26]:
function = udf(lambda x: int(x == "Home"), IntegerType())

uservisit = Window.partitionBy("userId").orderBy(desc("ts")).rangeBetween(Window.unboundedPreceding, 0)

cumsum = df.filter((df.page == "NextSong") | (df.page == "Home")).select("userID","page","ts")\
.withColumn("homevisit", function(col("page"))).withColumn("period", Fsum("homevisit").over(uservisit))

cumsum.filter((cumsum.page == "NextSong")).groupBy("userID","period").agg({"period": "count"})\
.agg({"count(period)": "avg"}).show()

+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+

